In [ ]:
from langchain.prompts import PromptTemplate
import shap

EXPLANATION_PROMPT = PromptTemplate(
    input_variables=[
        "out_of_stock_product",
        "recommended_product",
        "key_similarities",
        "ranking_reason"
    ],
    template="""
Tu es un assistant e-commerce chargé d'expliquer une recommandation produit à un client.

Produit initialement souhaité (en rupture) :
- {out_of_stock_product}

Produit recommandé en remplacement :
- {recommended_product}

Éléments de similarité clés :
{key_similarities}

Raisons principales du classement :
{ranking_reason}

Rédige une explication claire, courte et orientée client final :
- Ton rassurant
- Pas de jargon technique
- Mettre en avant la valeur pour le client
- 3 à 5 phrases maximum
"""
)


In [ ]:
explainer = shap.TreeExplainer(ranker)
shap_values = explainer.shap_values(X_topk) 

def get_top_shap_features(shap_values, feature_names, top_n=3):
    """
    Retourne les top_n features avec impact positif sur le score
    """
    top_features_list = []
    for i in range(shap_values.shape[0]):
        # prendre les indices des plus grandes valeurs SHAP
        top_idx = np.argsort(shap_values[i])[::-1][:top_n]
        top_features = [(feature_names[j], shap_values[i][j]) for j in top_idx]
        top_features_list.append(top_features)
    return top_features_list


In [ ]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    model="gpt-4o-mini",  # ou autre modèle
    temperature=0.4
)


In [ ]:
def generate_replacement_explanation_shap(
    out_of_stock_product: dict,
    recommended_product: dict,
    shap_features: list,
    ranking_score: float
) -> str:

    # Construire une version lisible des features SHAP
    key_similarities = []
    for feat, val in shap_features:
        # Interprétation simple côté client
        if "price" in feat.lower():
            key_similarities.append(f"- Gamme de prix similaire")
        elif "category" in feat.lower():
            key_similarities.append(f"- Même catégorie")
        elif "brand" in feat.lower():
            key_similarities.append(f"- Même marque")
        else:
            key_similarities.append(f"- {feat} similaire ou favorable")

    ranking_reason = (
        f"Ce produit a obtenu un score élevé ({ranking_score:.2f}) "
        "grâce à ses caractéristiques correspondant étroitement à vos préférences."
    )

    # Construire prompt
    prompt = EXPLANATION_PROMPT.format(
        out_of_stock_product=f"{out_of_stock_product['name']} ({out_of_stock_product['brand']})",
        recommended_product=f"{recommended_product['name']} ({recommended_product['brand']})",
        key_similarities="\n".join(key_similarities),
        ranking_reason=ranking_reason
    )

    # Appel LLM
    response = llm.invoke(prompt)
    return response.content


In [ ]:
out_of_stock_product = {
    "name": "Casque Bluetooth X200",
    "brand": "SoundMax"
}

recommended_product = {
    "name": "Casque Bluetooth Pro X300",
    "brand": "SoundMax"
}

model_features = {
    "same_category": "Oui",
    "price_similarity": "±5%",
    "attribute_similarity": "Réduction de bruit, autonomie équivalente"
}

ranking_score = 0.87  # score du LGBMRanker

explanation = generate_replacement_explanation(
    out_of_stock_product,
    recommended_product,
    model_features,
    ranking_score
)

print(explanation)


In [ ]:
import shap
import numpy as np
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain, SequentialChain
import hashlib
import json
import os


# 1Configuration LLM

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4)
CACHE_DIR = "./explanation_cache"
os.makedirs(CACHE_DIR, exist_ok=True)


# Prompts LangChain


# Node 1 : transforme SHAP en langage client-friendly
shap_prompt = PromptTemplate(
    input_variables=["shap_features"],
    template="""
Tu es un assistant e-commerce. Transforme les features SHAP suivantes en langage client-friendly :
{shap_features}

Renvoie une liste de points clairs, simples et rassurants.
"""
)

# Node 2 : génère l’explication finale pour le client
explanation_prompt = PromptTemplate(
    input_variables=["out_of_stock_product", "recommended_product", "key_similarities", "ranking_score"],
    template="""
Produit en rupture : {out_of_stock_product}
Produit recommandé : {recommended_product}
Points clés : {key_similarities}
Score : {ranking_score}

Rédige une explication courte, rassurante et orientée client.
"""
)


# Fonction utilitaire SHAP

def get_top_shap_features(shap_values, feature_names, top_n=3):
    top_features_list = []
    for i in range(shap_values.shape[0]):
        top_idx = np.argsort(shap_values[i])[::-1][:top_n]
        top_features = [(feature_names[j], shap_values[i][j]) for j in top_idx]
        top_features_list.append(top_features)
    return top_features_list


# Fonction pour générer explication via LangChain

def generate_explanation_chain(out_of_stock_product: str, recommended_product: str, shap_features: list, ranking_score: float):
    """
    Génère l’explication client en deux étapes :
    1️⃣ transformer SHAP en points clés
    2️⃣ générer texte final
    """
    # Cache key
    cache_key = hashlib.md5(
        json.dumps({
            "out": out_of_stock_product,
            "rec": recommended_product,
            "shap": shap_features,
            "score": ranking_score
        }, sort_keys=True).encode()
    ).hexdigest()
    cache_file = os.path.join(CACHE_DIR, f"{cache_key}.json")
    if os.path.exists(cache_file):
        with open(cache_file, "r") as f:
            return json.load(f)["explanation"]

    # --- Node 1 : SHAP -> points clés ---
    shap_chain = LLMChain(llm=llm, prompt=shap_prompt, output_key="key_similarities")
    key_similarities = shap_chain.run(shap_features=shap_features)

    # --- Node 2 : points clés -> explication finale ---
    explanation_chain = LLMChain(llm=llm, prompt=explanation_prompt, output_key="explanation_text")
    explanation_text = explanation_chain.run(
        out_of_stock_product=out_of_stock_product,
        recommended_product=recommended_product,
        key_similarities=key_similarities,
        ranking_score=ranking_score
    )

    # Sauvegarde cache
    with open(cache_file, "w") as f:
        json.dump({"explanation": explanation_text}, f, ensure_ascii=False)

    return explanation_text


# Pipeline complet : Ranker → SHAP → LangChain

def recommend_products_with_langchain(ranker, X_candidates: pd.DataFrame,
                                      candidate_products: list,
                                      out_of_stock_product: dict,
                                      top_k: int = 3):
    """
    Retourne top-k produits + explications générées via LangChain
    """
    # 1️⃣ Score top-k
    scores = ranker.predict(X_candidates)
    top_idx = np.argsort(scores)[::-1][:top_k]
    top_products = [candidate_products[i] for i in top_idx]
    top_scores = [scores[i] for i in top_idx]

    # 2️⃣ SHAP
    explainer = shap.TreeExplainer(ranker)
    shap_values = explainer.shap_values(X_candidates)
    top_shap_features_list = get_top_shap_features(shap_values, X_candidates.columns.tolist(), top_n=3)
    top_shap_features = [top_shap_features_list[i] for i in top_idx]

    # 3️⃣ LangChain pour explications
    explanations = []
    for prod, shap_feat, score in zip(top_products, top_shap_features, top_scores):
        out_product_str = f"{out_of_stock_product['name']} ({out_of_stock_product['brand']})"
        rec_product_str = f"{prod['name']} ({prod['brand']})"
        explanation = generate_explanation_chain(out_product_str, rec_product_str, shap_feat, score)
        explanations.append(explanation)

    # 4️⃣ Retour
    result = []
    for prod, score, exp in zip(top_products, top_scores, explanations):
        result.append({
            "product": prod,
            "score": score,
            "explanation": exp
        })

    return result
